In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

## Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
 # create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect = True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
measurements = Base.classes.measurement
stations = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

## Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
recent_date = session.query(measurements.date).order_by(measurements.date.desc()).first()
recent_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
year_from_recent = dt.date(2017,8,23) - dt.timedelta(days=365)
year_from_recent

In [ ]:
# Perform a query to retrieve the data and precipitation scores
sel = [measurements.date,measurements.prcp]
dates_prcp = session.query(*sel).\
            filter(measurements.date > '2016-08-22').\
            filter(measurements.date < '2017-08-22').\
            order_by(measurements.date).all()
dates_prcp

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
measurements_df = pd.DataFrame(dates_prcp, columns=['Date', 'Precipitation Score'])
measurements_df.set_index('Date', inplace=True)
measurements_df

In [ ]:
# Sort the dataframe by date
measurements_df = measurements_df.sort_index()
measurements_df

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data

measurements_df.plot(rot=90, figsize=(12,6))
plt.title("Precipitation 8/23/2016 - 8/23/2017")
plt.ylabel("Inches")
plt.xlabel("Date")
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
measurements_df.describe()

## Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
total_stations = session.query(stations.station).count()
total_stations

In [ ]:
# Design a query to find the most active stations 
#(i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
active_stations = session.query(measurements.station, func.count(measurements.station))\
                                .group_by(measurements.station)\
                                .order_by(func.count(measurements.station).desc())\
                                .all()
active_stations

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active_station_id = active_stations[0][0]
most_active_station_id

In [ ]:
# Query the min temperature
min_temp = session.query(func.min(measurements.tobs))\
                    .filter(measurements.station == most_active_station_id)\
                    .all()
min_temp = min_temp[0][0]
min_temp

In [ ]:
# Query the max temperature
max_temp = session.query(func.max(measurements.tobs))\
                    .filter(measurements.station == most_active_station_id)\
                    .all()
max_temp = max_temp[0][0]
max_temp

In [ ]:
# Query the avg temperature
avg_temp = session.query(func.avg(measurements.tobs))\
                    .filter(measurements.station == most_active_station_id)\
                    .all()
avg_temp = avg_temp[0][0]
avg_temp

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
temp_freq = session.query(measurements.date,measurements.station,measurements.tobs)\
            .filter(measurements.station == most_active_station_id)\
            .filter(measurements.date > '2016-08-22')\
            .filter(measurements.date < '2017-08-22')\
            .order_by(measurements.date).all()
temp_freq

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
active_station_temps_df = pd.DataFrame(temp_freq, columns=['Date', 'Station', 'Temperature'])
active_station_temps_df.set_index('Date', inplace=True)
active_station_temps_df

In [ ]:
# Plotting Frequency v Temperature at most active station
plt.hist(active_station_temps_df['Temperature'], bins=12)
plt.title(f'Temperature 8/23/2016 - 8/23/2017 at Station {most_active_station_id}')
plt.xlabel('Temperature(F)')
plt.ylabel('Frequency')
plt.show()

##  Close session

In [ ]:
# Close Session
session.close()